In [57]:
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn as sk

In [58]:
def clean_data (df):
    # Fare
    df.Fare = df.Fare.map(lambda x: np.nan if x == 0 else x)
    # for each pclass
    classmeans = pd.pivot_table(df, 'Fare', 'Pclass', aggfunc='mean')
    df.Fare = df[['Fare', 'Pclass']].apply(lambda x: classmeans[x['Pclass']] if pd.isnull(x['Fare']) else x['Fare'], axis=1)
    # Age
    media = np.mean(df.Age)
    df.Age = df.Age.fillna(media)
    # Cabin
    df.Cabin = df.Cabin.fillna('Unknown')
    # Embarked 
    moda = df.Embarked.mode()[0]
    df.Embarked.fillna(moda, inplace=True)
    
    return df

In [59]:
def read_csv (n_bins=0):
    df = pd.read_csv('Titanic/train.csv')
    tdf = pd.read_csv('Titanic/test.csv')
    if n_bins == 0:
        return [clean_data(df), clean_data(tdf)]
    df = clean_data(df)
    tdf = clean_data(tdf)
    
    bins_binned_fare = pd.qcut(df.Fare, n_bins, retbins=True)
    bins = bins_binned_fare[1]
    
    df.Fare = bins_binned_fare[0]
    tdf.Fare = pd.cut(tdf.Fare, bins)
    
    #discretise age
    bins_binned_age = pd.qcut(df.Age, n_bins, retbins=True)
    bins = bins_binned_age[1]
    
    df.Age = bins_binned_age[0]
    tdf.Age = pd.cut(tdf.Age, bins)
    
    predictiondf = pd.DataFrame(tdf['PassengerID'])
    predictiondf['Survived'] = [0 for x in range(list(tdf))]
    predictiondf.to_csv('prediction.csv', index=False)
    return [df, tdf]

In [67]:
df = read_csv(n_bins=10)[0]
tdf = read_csv(n_bins=10)[1]

df = df[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
data_type_dict = {'Survived':'nominal', 'Pclass':'ordinal', 'Sex':'nominal', 
                'Age':'ordinal', 'SibSp':'ordinal', 'Parch':'ordinal', 
                'Fare':'ordinal', 'Embarked':'nominal'}


ValueError: Bin edges must be unique: array([  0.42      ,  16.        ,  20.        ,  24.        ,
        28.        ,  29.69911765,  29.69911765,  32.5       ,
        38.        ,  47.        ,  80.        ])